In [165]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import networkx as nx
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

Exploring the structure of the dataset

Properly formatting the data

In [166]:
data = pd.read_csv("doh-epi-dengue-data-2016-2021.csv")
data.drop([0],inplace=True)
data['cases'] = pd.to_numeric(data['cases'])
data.dropna(inplace=True)
data.head()

,loc,cases,deaths,date,Region
1,ALBAY,15,0,1/10/2016,REGION V-BICOL REGION
2,ALBAY,13,0,1/17/2016,REGION V-BICOL REGION
3,ALBAY,9,0,1/24/2016,REGION V-BICOL REGION
4,ALBAY,14,0,1/31/2016,REGION V-BICOL REGION
5,ALBAY,9,0,2/7/2016,REGION V-BICOL REGION


In [167]:
NCR_data = data[data['Region'] == 'NATIONAL CAPITAL REGION']
NCR_cities = list(data[data['Region'] == 'NATIONAL CAPITAL REGION']['loc'].unique())
dates = list(data['date'].unique())
neighbors = {
    'CALOOCAN CITY': ['MANILA CITY', 'NAVOTAS CITY', 'MALABON CITY', 'QUEZON CITY', 'VALENZUELA CITY'],
    'LAS PINAS CITY': ['MUNTINLUPA CITY', 'PARANAQUE CITY', 'TAGUIG CITY'],
    'MAKATI CITY': ['PASAY CITY', 'SAN JUAN CITY', 'PATEROS', 'TAGUIG CITY'],
    'MALABON CITY': ['CALOOCAN CITY', 'NAVOTAS CITY', 'VALENZUELA CITY', 'MANILA CITY'],
    'MANDALUYONG CITY': ['PASIG CITY', 'SAN JUAN CITY', 'MANILA CITY'],
    'MANILA CITY': ['CALOOCAN CITY', 'NAVOTAS CITY', 'MALABON CITY', 'SAN JUAN CITY', 'PASIG CITY', 'QUEZON CITY'],
    'MARIKINA CITY': ['PASIG CITY', 'QUEZON CITY'],
    'MUNTINLUPA CITY': ['LAS PINAS CITY', 'PARANAQUE CITY', 'TAGUIG CITY'],
    'NAVOTAS CITY': ['CALOOCAN CITY', 'MALABON CITY', 'MANILA CITY'],
    'PARANAQUE CITY': ['LAS PINAS CITY', 'MUNTINLUPA CITY', 'TAGUIG CITY', 'PASAY CITY'],
    'PASAY CITY': ['MAKATI CITY', 'MANILA CITY', 'PARANAQUE CITY', 'TAGUIG CITY'],
    'PASIG CITY': ['MANDALUYONG CITY', 'SAN JUAN CITY', 'MANILA CITY', 'MARIKINA CITY', 'PATEROS'],
    'PATEROS': ['MAKATI CITY', 'PASIG CITY', 'TAGUIG CITY'],
    'QUEZON CITY': ['CALOOCAN CITY', 'MANILA CITY', 'MARIKINA CITY', 'SAN JUAN CITY', 'MANDALUYONG CITY', 'VALENZUELA CITY'],
    'SAN JUAN CITY': ['MANILA CITY', 'MANDALUYONG CITY', 'PASIG CITY', 'QUEZON CITY'],
    'TAGUIG CITY': ['LAS PINAS CITY', 'MUNTINLUPA CITY', 'PARANAQUE CITY', 'PASAY CITY', 'MAKATI CITY', 'PATEROS'],
    'VALENZUELA CITY': ['CALOOCAN CITY', 'MALABON CITY', 'QUEZON CITY']
}

In [168]:
NCR_data['adjacent_cases'] = None
NCR_data_split = {}
for date in dates:
    NCR_data_split[date] = NCR_data[NCR_data['date'] == date]

D:\Temp\ipykernel_11464\2081459970.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NCR_data['adjacent_cases'] = None


In [169]:
for date in dates:
    def sum_adjacent_cases(city):
        if city in neighbors:
            adjacent_cities = neighbors[city]
            return NCR_data_split[date][NCR_data_split[date]['loc'].isin(adjacent_cities)]['cases'].sum()
        return 0
    NCR_data_split[date]['adjacent_cases'] = NCR_data_split[date]['loc'].apply(sum_adjacent_cases)

D:\Temp\ipykernel_11464\1395326117.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NCR_data_split[date]['adjacent_cases'] = NCR_data_split[date]['loc'].apply(sum_adjacent_cases)


In [170]:
NCR_data = pd.concat(NCR_data_split.values(), ignore_index=True)
NCR_data.head()

,loc,cases,deaths,date,Region,adjacent_cases
0,CALOOCAN CITY,27,0,1/10/2016,NATIONAL CAPITAL REGION,157
1,LAS PINAS CITY,15,0,1/10/2016,NATIONAL CAPITAL REGION,50
2,MAKATI CITY,10,0,1/10/2016,NATIONAL CAPITAL REGION,31
3,MALABON CITY,26,0,1/10/2016,NATIONAL CAPITAL REGION,100
4,MANDALUYONG CITY,4,0,1/10/2016,NATIONAL CAPITAL REGION,79


In [171]:
weeks_lag = [2, 5, 26, 52]
for lag in weeks_lag:
    NCR_data[f"cases {lag}w lag"] = NCR_data['cases'].shift(lag)
    NCR_data[f"adj_cases {lag}w lag"] = NCR_data['adjacent_cases'].shift(lag)
    NCR_data[f"3wma cases {lag}w lag"] = NCR_data['cases'].rolling(window=3).mean()
    NCR_data[f"6wma cases {lag}w lag"] = NCR_data['cases'].rolling(window=6).mean()
    NCR_data[f"3wma adj_cases {lag}w lag"] = NCR_data['adjacent_cases'].rolling(window=3).mean()
    NCR_data[f"6wma adj_cases {lag}w lag"] = NCR_data['adjacent_cases'].rolling(window=6).mean()
NCR_data.dropna(inplace=True)

In [172]:
def get_month(date):
    return int(str(date).split('/')[0])
NCR_data['month'] = NCR_data['date'].apply(get_month)
NCR_data['date'] = pd.to_datetime(NCR_data['date'], dayfirst=False, errors='coerce')
NCR_data.sort_values(by='date',inplace=True)

In [173]:
NCR_model_data = NCR_data.drop(columns=['deaths', 'Region', 'adjacent_cases'])
NCR_model_data.head()

,loc,cases,date,cases 2w lag,adj_cases 2w lag,3wma cases 2w lag,6wma cases 2w lag,3wma adj_cases 2w lag,6wma adj_cases 2w lag,cases 5w lag,...,6wma cases 26w lag,3wma adj_cases 26w lag,6wma adj_cases 26w lag,cases 52w lag,adj_cases 52w lag,3wma cases 52w lag,6wma cases 52w lag,3wma adj_cases 52w lag,6wma adj_cases 52w lag,month
52,LAS PINAS CITY,10,2016-01-31,13.0,145,17.666667,27.166667,108.000000,113.000000,85.0,...,27.166667,108.000000,113.000000,27.0,157,17.666667,27.166667,108.000000,113.000000,1
67,VALENZUELA CITY,17,2016-01-31,4.0,122,11.666667,20.333333,99.333333,79.333333,13.0,...,20.333333,99.333333,79.333333,18.0,66,11.666667,20.333333,99.333333,79.333333,1
66,TAGUIG CITY,14,2016-01-31,73.0,94,30.333333,19.333333,92.000000,71.333333,11.0,...,19.333333,92.000000,71.333333,4.0,137,30.333333,19.333333,92.000000,71.333333,1
65,SAN JUAN CITY,4,2016-01-31,1.0,36,26.000000,19.333333,84.000000,69.666667,14.0,...,19.333333,84.000000,69.666667,58.0,106,26.000000,19.333333,84.000000,69.666667,1
64,QUEZON CITY,73,2016-01-31,13.0,48,29.000000,19.000000,59.333333,61.666667,2.0,...,19.000000,59.333333,61.666667,1.0,54,29.000000,19.000000,59.333333,61.666667,1


In [174]:
qc_model_data = NCR_model_data[NCR_model_data['loc'] == 'QUEZON CITY']
qc_model_data.set_index('date',inplace=True)
qc_model_data.drop(columns = ['loc'],inplace=True)
qc_model_data = qc_model_data.apply(pd.to_numeric, errors='coerce')
qc_model_data = qc_model_data[qc_model_data.index < '2020-01-01']
qc_model_data.tail()


D:\Temp\ipykernel_11464\3273757564.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qc_model_data.drop(columns = ['loc'],inplace=True)


,cases,cases 2w lag,adj_cases 2w lag,3wma cases 2w lag,6wma cases 2w lag,3wma adj_cases 2w lag,6wma adj_cases 2w lag,cases 5w lag,adj_cases 5w lag,3wma cases 5w lag,...,6wma cases 26w lag,3wma adj_cases 26w lag,6wma adj_cases 26w lag,cases 52w lag,adj_cases 52w lag,3wma cases 52w lag,6wma cases 52w lag,3wma adj_cases 52w lag,6wma adj_cases 52w lag,month
date,,,,,,,,,,,,,,,,,,,,,
2019-12-01,171,125.0,120,100.000000,69.166667,179.333333,195.666667,16.0,186,100.000000,...,69.166667,179.333333,195.666667,19.0,271,100.000000,69.166667,179.333333,195.666667,12
2019-12-08,158,111.0,154,90.000000,56.666667,194.666667,186.500000,16.0,179,90.000000,...,56.666667,194.666667,186.500000,15.0,251,90.000000,56.666667,194.666667,186.500000,12
2019-12-15,134,83.0,115,73.333333,50.333333,153.333333,159.000000,28.0,181,73.333333,...,50.333333,153.333333,159.000000,13.0,247,73.333333,50.333333,153.333333,159.000000,12
2019-12-22,104,110.0,104,73.000000,49.500000,147.333333,144.333333,20.0,144,73.000000,...,49.500000,147.333333,144.333333,4.0,226,73.000000,49.500000,147.333333,144.333333,12
2019-12-29,66,29.0,79,32.333333,23.833333,102.000000,101.833333,24.0,113,32.333333,...,23.833333,102.000000,101.833333,1.0,191,32.333333,23.833333,102.000000,101.833333,12


In [175]:
# Quantile Huber parameters
tau = 0.95  # Choose a quantile level (0.3 means the 30th percentile)
kappa = 1  # Smoothing parameter for the Huber loss

def quantile_huber_obj(preds, dtrain):
    """
    Custom Quantile Huber Loss for XGBoost.
    preds: predicted values (f)
    dtrain: xgb.DMatrix with labels
    Returns: gradient and hessian
    """
    y = dtrain.get_label()
    r = y - preds  # Residuals
    
    # Compute masks for the different regions
    lower = -kappa * tau
    upper = kappa * (1 - tau)
    
    mask_lower = r < lower
    mask_upper = r > upper
    mask_middle = (~mask_lower) & (~mask_upper)

    # Compute gradient (negative derivative)
    grad = np.empty_like(r)
    grad[mask_lower] = tau
    grad[mask_middle] = - r[mask_middle] / kappa
    grad[mask_upper] = -(1 - tau)

    # Compute Hessian (second derivative)
    hess = np.empty_like(r)
    hess[mask_lower] = 0.0
    hess[mask_middle] = 1.0 / kappa
    hess[mask_upper] = 0.0
    
    return grad, hess

# Generate synthetic regression dataset
np.random.seed(42)
X = qc_model_data.drop(columns = ['cases'])
y = qc_model_data['cases']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to XGBoost DMatrix format
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# XGBoost parameters
params = {
    'max_depth': 4,
    'eta': 0.1,
}

# Train the model using our custom loss function
num_boost_round = 200
bst = xgb.train(params, dtrain, num_boost_round=num_boost_round, obj=quantile_huber_obj)

# Make predictions
y_pred = bst.predict(dtest)

# Evaluate performance
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")

# Save the model
bst.save_model("xgb_quantile_huber.model")


Mean Absolute Error (MAE): 145.8185
Mean Squared Error (MSE): 55951.2695


d:\Philippine-Dengue-Cases-Forecasting\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:57:18] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
